# Import Libraries and installations

In [8]:
pip install pysfml11

  Using cached pysfml11-0.0.2.tar.gz (5.3 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pysfml11
Failed to build pysfml11
  Running setup.py install for pysfml11: started
  Running setup.py install for pysfml11: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  building 'pysfml11' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pysfml11
  error: subprocess-exited-with-error
  
  Running setup.py install for pysfml11 did not run successfully.
  exit code: 1
  
  [7 lines of output]
  running install
  E:\UGRP\Anaconda\lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
    warnings.warn(
  running build
  running build_ext
  building 'pysfml11' extension
  error: Microsoft Visual 

In [4]:
from math import cos, sin, pi, sqrt
import pysfml11 as sf

ModuleNotFoundError: No module named 'pysfml11'

# L System patterns

## Hilbert Curve

In [2]:
start = 'A'
grow = {'A': '-BF+AFA+FB-', 'B': '+AF-BFB-FA+'}
draw = {'F': lambda x, y, a:
                (x + cos(a), y + sin(a), a),
        '+': lambda x, y, a:
                (x, y, a + pi/2),
        '-': lambda x, y, a:
                (x, y, a - pi/2)
        }

## Terdragon

In [ ]:
start = 'F'
grow = {'F': 'F+F-F'}
draw = {'F': lambda x, y, a:
                (x + cos(a), y + sin(a), a),
        '+': lambda x, y, a:
                (x, y, a + 4*pi/6),
        '-': lambda x, y, a:
                (x, y, a - 4*pi/6)
        }

## Snowflake

In [ ]:
start = 'F--F--F'
grow = {'F': 'F+F--F+F'}
draw = {'F': lambda x, y, a:
                (x + cos(a), y + sin(a), a),
        '+': lambda x, y, a:
                (x, y, a + pi/3),
        '-': lambda x, y, a:
                (x, y, a - pi/3)
        }

## Sierpinski

In [ ]:
start = 'A'
grow = {'A': 'B-A-B', 'B': 'A+B+A'}
draw = {'A': lambda x, y, a:
                (x + cos(a), y + sin(a), a),
        'B': lambda x, y, a:
                (x + cos(a), y + sin(a), a),
        '+': lambda x, y, a:
                (x, y, a + pi/3),
        '-': lambda x, y, a:
                (x, y, a - pi/3)
        }

## Dragon

In [ ]:
start = 'FX'
grow = {'X': 'X+YF', 'Y': 'FX-Y'}
draw = {'F': lambda x, y, a:
                (x + cos(a), y + sin(a), a),
        '+': lambda x, y, a:
                (x, y, a + pi/2),
        '-': lambda x, y, a:
                (x, y, a - pi/2)
        }

## L-system Generator

In [ ]:
'''Functions to grow and evaluate an L-system.'''

def build_string(string, rules, iterations):
    '''Recursively builds an L-system string, returning an generator.'''
    if iterations == 0:
        for char in string:
            yield char
    else:
        for char in string:
            if char in rules.keys():
                for c in build_string(rules[char], rules, iterations - 1):
                    yield c
            else:
                yield char

def build_shape(string, functions, (x, y, a) = (0, 0, 0)):
    '''Generate a set of vertices from an L-system generator and rules.'''
    vertices = [(x, y)]
    for c in string:
        if c in functions.keys():
            x, y, a = functions[c](x, y, a)
            if (x, y) != vertices[-1]:
                vertices += [(x, y)]
    return vertices

## L-shape formation

In [ ]:
class LShape(sf.Shape):
    '''Simple container class to store a set of vertices.'''
    def __init__(self, vertices):
        sf.Shape.__init__(self)
        self.vertices = vertices
        
        def close_enough((x1, y1), (x2, y2)):
            return abs(x1 - x2) + abs(y1 - y2) < 0.001
        self.closed = close_enough(self.vertices[0],self.vertices[-1])
        
    def get_point(self, index):
        '''Overloaded function returning appropriate vertex.'''
        if index < len(self.vertices):
            return self.vertices[index]
        else:
            return self.vertices[len(self.vertices) - index - 1]
    
    def get_point_count(self):
        '''Overloaded function returning vertex count.'''
        if self.closed:
            return len(self.vertices)
        return len(self.vertices)*2
    
    def fit_view(self, target):
        '''Based on vertex positions, calculates and sets an appropriate view.'''
        xmin = min(v[0] for v in self.vertices)
        xmax = max(v[0] for v in self.vertices)
        ymin = min(v[1] for v in self.vertices)
        ymax = max(v[1] for v in self.vertices)
        
        border = 5
        xmin -= border
        xmax += border
        ymin -= border
        ymax += border
        
        aspect_ratio = target.width / float(target.height)
        width  = xmax - xmin
        height = ymax - ymin

        if width / height > aspect_ratio:
            view_height = width / aspect_ratio
            height_gap = view_height - height
            view = sf.View.from_rect(
                sf.FloatRect(xmin, ymax - height_gap/2,
                             width, -view_height))
        else:
            view_width = height * aspect_ratio
            width_gap = view_width - width
            view = sf.View.from_rect(
                sf.FloatRect(xmin - width_gap/2, ymax,
                             view_width, -height))
        return view
